In [1]:
import sys
import pandas as pd
import numpy as np
import math
import heapq
import itertools

In [2]:
def get_route(start, end, cost):
    
    """
    Find shortest driving route between start city and end city
    based on a cost function.

    1. Your function should return a dictionary having the following keys:
        -"route-taken" : a list of pairs of the form (next-stop, segment-info), where
           next-stop is a string giving the next stop in the route, and segment-info is a free-form
           string containing information about the segment that will be displayed to the user.
           (segment-info is not inspected by the automatic testing program).
        -"total-segments": an integer indicating number of segments in the route-taken
        -"total-miles": a float indicating total number of miles in the route-taken
        -"total-hours": a float indicating total amount of time in the route-taken
        -"total-delivery-hours": a float indicating the expected (average) time 
                                   it will take a delivery driver who may need to return to get a new package
    2. Do not add any extra parameters to the get_route() function, or it will break our grading and testing code.
    3. Please do not use any global variables, as it may cause the testing code to fail.
    4. You can assume that all test cases will be solvable.
    5. The current code just returns a dummy solution.
    """

    route_taken = [("Martinsville,_Indiana","IN_37 for 19 miles"),
                   ("Jct_I-465_&_IN_37_S,_Indiana","IN_37 for 25 miles"),
                   ("Indianapolis,_Indiana","IN_37 for 7 miles")]
    
    return {"total-segments" : len(route_taken), 
            "total-miles" : 51., 
            "total-hours" : 1.07949, 
            "total-delivery-hours" : 1.1364, 
            "route-taken" : route_taken}

In [ ]:
# Please don't modify anything below this line
#
if __name__ == "__main__":
    if len(sys.argv) != 4:
        raise(Exception("Error: expected 3 arguments"))

    (_, start_city, end_city, cost_function) = sys.argv
    if cost_function not in ("segments", "distance", "time", "delivery"):
        raise(Exception("Error: invalid cost function"))

    result = get_route(start_city, end_city, cost_function)

    # Pretty print the route
    print("Start in %s" % start_city)
    for step in result["route-taken"]:
        print("   Then go to %s via %s" % step)

    print("\n          Total segments: %4d" % result["total-segments"])
    print("             Total miles: %8.3f" % result["total-miles"])
    print("             Total hours: %8.3f" % result["total-hours"])
    print("Total hours for delivery: %8.3f" % result["total-delivery-hours"])




In [5]:
def parse_map(filename):
    with open(filename, "r") as f:
        return([line for line in f.read().split("\n")])

road_seg = parse_map("road-segments.txt")
city_gps = parse_map("city-gps.txt")

In [2]:
road_seg = pd.read_csv("road-segments.txt",
                      names = ['city_1','city_2','length','speed','highway'],
                      usecols = [x for x in range(0,5)],
                      dtype = {'city':str,'city_2':str,'length':int,'speed':int,'highway':str},
                      sep = ' ',
                      header = None)
road_seg

,city_1,city_2,length,speed,highway
0,"Y_City,_Arkansas","Acorn,_Arkansas",15,45,US_71/270
1,"Y_City,_Arkansas","Greenwood,_Arkansas",46,45,US_71
2,"Y_City,_Arkansas","Hot_Springs,_Arkansas",70,45,US_270
3,"Abbot_Village,_Maine","Bingham,_Maine",24,45,ME_16
4,"Abbot_Village,_Maine","Guilford,_Maine",4,45,ME_6/15/16
...,...,...,...,...,...
12033,"Winchester,_Wisconsin","Winnebago,_Wisconsin",12,45,WI_110
12034,"Windom,_Minnesota","Worthington,_Minnesota",31,45,MN_60
12035,"Windsor,_North_Carolina","Winton,_North_Carolina",27,52,US_13
12036,"Woodland,_California","Yuba_City,_California",36,45,CA_99_&_113


In [3]:
road_seg['time'] = round(road_seg.length/road_seg.speed,5)
road_seg

,city_1,city_2,length,speed,highway,time
0,"Y_City,_Arkansas","Acorn,_Arkansas",15,45,US_71/270,0.33333
1,"Y_City,_Arkansas","Greenwood,_Arkansas",46,45,US_71,1.02222
2,"Y_City,_Arkansas","Hot_Springs,_Arkansas",70,45,US_270,1.55556
3,"Abbot_Village,_Maine","Bingham,_Maine",24,45,ME_16,0.53333
4,"Abbot_Village,_Maine","Guilford,_Maine",4,45,ME_6/15/16,0.08889
...,...,...,...,...,...,...
12033,"Winchester,_Wisconsin","Winnebago,_Wisconsin",12,45,WI_110,0.26667
12034,"Windom,_Minnesota","Worthington,_Minnesota",31,45,MN_60,0.68889
12035,"Windsor,_North_Carolina","Winton,_North_Carolina",27,52,US_13,0.51923
12036,"Woodland,_California","Yuba_City,_California",36,45,CA_99_&_113,0.80000


In [4]:
road_seg['prob_overspeed'] = 0
road_seg.loc[road_seg['speed']>=50,'prob_overspeed'] = np.tanh(road_seg.loc[road_seg['speed']>=50]['length']/1000)

In [5]:
road_seg

,city_1,city_2,length,speed,highway,time,prob_overspeed
0,"Y_City,_Arkansas","Acorn,_Arkansas",15,45,US_71/270,0.33333,0.000000
1,"Y_City,_Arkansas","Greenwood,_Arkansas",46,45,US_71,1.02222,0.000000
2,"Y_City,_Arkansas","Hot_Springs,_Arkansas",70,45,US_270,1.55556,0.000000
3,"Abbot_Village,_Maine","Bingham,_Maine",24,45,ME_16,0.53333,0.000000
4,"Abbot_Village,_Maine","Guilford,_Maine",4,45,ME_6/15/16,0.08889,0.000000
...,...,...,...,...,...,...,...
12033,"Winchester,_Wisconsin","Winnebago,_Wisconsin",12,45,WI_110,0.26667,0.000000
12034,"Windom,_Minnesota","Worthington,_Minnesota",31,45,MN_60,0.68889,0.000000
12035,"Windsor,_North_Carolina","Winton,_North_Carolina",27,52,US_13,0.51923,0.026993
12036,"Woodland,_California","Yuba_City,_California",36,45,CA_99_&_113,0.80000,0.000000


In [10]:
city_gps = pd.read_csv("city-gps.txt",
                      names = ['city','lat','long'],
                      usecols = [x for x in range(0,3)],
                      dtype = {'city':str,'lat':float,'long':float},
                      sep = ' ',
                      header = None)
city_gps

,city,lat,long
0,"Y_City,_Arkansas",34.735104,-94.073257
1,"Abbot_Village,_Maine",45.197684,-69.458819
2,"Abbotsford,_Wisconsin",44.946356,-90.315969
3,"Abbott,_New_Mexico",36.305585,-104.258870
4,"Abbyville,_Kansas",37.970847,-98.204230
...,...,...,...
5473,"Zilwaukee,_Michigan",43.476414,-83.920529
5474,"Zion_Crossroads,_Virginia",37.970974,-78.219726
5475,"Zionsville,_Indiana",39.950873,-86.261937
5476,"Zolfo_Springs,_Florida",27.493372,-81.795916


In [11]:
def successors(city):

    #next_city = road_seg.city_2
    next_city_details = []
    for i in range(road_seg.shape[0]):
        if road_seg.city_1[i] == city:
            next_city_details.append([road_seg.city_2[i],road_seg.length[i],road_seg.speed[i],road_seg.highway[i],road_seg.time[i],road_seg.prob_overspeed[i]])
        elif road_seg.city_2[i] == city:
            next_city_details.append([road_seg.city_1[i],road_seg.length[i],road_seg.speed[i],road_seg.highway[i],road_seg.time[i],road_seg.prob_overspeed[i]])
    return next_city_details


In [12]:
p=successors('Winchester,_Wisconsin')
p
#next_city,length,speed,highway,time,prob

[['Fremont,_Wisconsin', 12, 45, 'WI_110', 0.26667, 0.0],
 ['Neenah,_Wisconsin', 8, 45, 'WI_150', 0.17778, 0.0],
 ['Winnebago,_Wisconsin', 12, 45, 'WI_110', 0.26667, 0.0]]

In [13]:
0.621371*6371

3958.754641

In [14]:
#https://stackoverflow.com/questions/36873187/heuristic-for-an-a-path-finding-gps?rq=1
#This function calculates the heuristics using haversine formula
def get_heuristic(lat_1,long_1,lat_2,long_2):
    rad = 0.621371*6371 #in miles
    dLat = (lat_2-lat_1)*(math.pi/180)
    dLon = (long_2-long_1)*(math.pi/180)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat_1*(math.pi/180)) * math.cos(lat_2*(math.pi/180)) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = rad * c; 
    return d

In [15]:
city_gps[city_gps.city == 'Fremont,_Wisconsin']['lat'].iloc[0]

44.2597027

In [46]:
road_seg.length.sum()/road_seg.shape[0]

24.246718724040537

In [47]:
road_seg.length.mean()

24.246718724040537

In [16]:



################# segment #################

def route_seg(start, end, cost):
    fringe = []
    visited_cities = []
    visited_cities.append(start)
    if end in city_gps.city.unique():
        lat2 = city_gps[city_gps.city == end]['lat'].iloc[0]
        long2 = city_gps[city_gps.city == end]['long'].iloc[0]

    
    heapq.heappush(fringe,(0,start,0,0,0,[])) #no of road seg, curr city, length,time, extra delivery time, path
    while len(fringe) != 0:
        
        (segment, start_city, distance, t_trip, delivery, path) = heapq.heappop(fringe)
        
        if start_city == end:

            return {"total-segments" : len(path), 
            "total-miles" : float(distance), 
            "total-hours" : t_trip, 
            "total-delivery-hours" : round(delivery,4),
            "route-taken" : path}
            
        #visited_cities.append(start_city)
        #route_taken=[]
        for (next_city,length,speed,highway,t_road,prob) in successors(start_city):
            try:
                #lat2 = city_gps[city_gps.city == end]['lat'].iloc[0]
                #long2 = city_gps[city_gps.city == end]['long'].iloc[0]
                lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                h =  get_heuristic(lat1,long1,lat2,long2)
            except:
                h = 0
            '''
            if next_city == end:
                
                try:
                    lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                    long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                    h =  get_heuristic(lat1,long1,lat2,long2)
                except:
                    h = 0
            
                d = distance + length
                t = t_trip + t_road
                del_time = round(delivery + (t_road + prob*2*(t_trip+t_road)),4)
                path.append((next_city,highway+" for "+str(length)+" miles"))

                return {"total-segments" : len(path), 
                "total-miles" : float(d), 
                "total-hours" : t, 
                "total-delivery-hours" : del_time, 
                "route-taken" : path}
            
            '''
            if next_city not in visited_cities:
                #print(next_city)
                #segment = len(path)
                #cost_seg = segment + 1
                cost_seg = len(path) + h/(road_seg.length.mean()) #heuristic
                del_time = round(delivery + t_road + prob*2*(t_trip+t_road),4)
                #del_time = round(t_road + prob*2*(t_trip+t_road),4)
                #path.append([next_city,length,speed,highway,t_road,prob])
                movement = (next_city,highway+" for "+str(length)+" miles")
                #path.append([next_city,length,highway,t_road,del_time])
                heapq.heappush(fringe,(cost_seg,next_city,distance+length,t_trip+t_road,del_time,path+[movement]))#[(next_city,length,speed,highway,t_road,prob)]))
                visited_cities.append(next_city)
                

In [17]:
for i in ['Bloomington,_Indiana', 'Jct_I-465_&_IN_37_S,_Indiana']:
    if i in city_gps.city.unique():
        print('yes')
    else:
        print('no')

yes
no


In [18]:



################# distance #################

def route_dist(start, end, cost):
    fringe = []
    visited_cities = []
    visited_cities.append(start)
    if end in city_gps.city.unique():
        lat2 = city_gps[city_gps.city == end]['lat'].iloc[0]
        long2 = city_gps[city_gps.city == end]['long'].iloc[0]

    heapq.heappush(fringe,(0,start,0,0,0,[])) #no of road seg, curr city, length,time, extra delivery time, path
    while len(fringe) != 0:
        
        
        (heuristic, start_city, distance, t_trip, delivery, path) = heapq.heappop(fringe)
        '''
        if start_city == end:
            #return path
            
            d = distance
            t = t_trip
            del_time = delivery
            #path.append((next_city,highway+" for "+str(length)+" miles"))

            return {"total-segments" : len(path), 
            "total-miles" : float(d), 
            "total-hours" : t, 
            "total-delivery-hours" : del_time, 
            "route-taken" : path}
        '''
        #visited_cities.append(start_city)
        #route_taken=[]
        for (next_city,length,speed,highway,t_road,prob) in successors(start_city):
            try:
                lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                h =  get_heuristic(lat1,long1,lat2,long2)
            except:
                h = 0
            
            
            if next_city == end:
                '''
                try:
                    lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                    long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                    h =  get_heuristic(lat1,long1,lat2,long2)
                except:
                    h = 0
                '''
                d = distance + length
                t = t_trip + t_road
                del_time = round(delivery + (t_road + prob*2*(t_trip+t_road)),4)
                path.append((next_city,highway+" for "+str(length)+" miles"))

                return {"total-segments" : len(path), 
                "total-miles" : float(d), 
                "total-hours" : t, 
                "total-delivery-hours" : del_time, 
                "route-taken" : path}
            
            
            if next_city not in visited_cities:
                #print(next_city)
                #segment = len(path)
                #cost_seg = segment + 1
                cost_dist = distance + h #heuristic
                del_time = round(delivery + t_road + prob*2*(t_trip+t_road),4)
                #del_time = round(t_road + prob*2*(t_trip+t_road),4)
                #path.append([next_city,length,speed,highway,t_road,prob])
                movement = (next_city,highway+" for "+str(length)+" miles")
                #path.append([next_city,length,highway,t_road,del_time])
                heapq.heappush(fringe,(cost_dist,next_city,distance+length,t_trip+t_road,del_time,path+[movement]))#[(next_city,length,speed,highway,t_road,prob)]))
                visited_cities.append(next_city)
                

In [19]:



################# time #################

def route_time(start, end, cost):
    fringe = []
    visited_cities = []
    visited_cities.append(start)
    if end in city_gps.city.unique():
        lat2 = city_gps[city_gps.city == end]['lat'].iloc[0]
        long2 = city_gps[city_gps.city == end]['long'].iloc[0]

    heapq.heappush(fringe,(0,start,0,0,0,[])) #no of road seg, curr city, length,time, extra delivery time, path
    while len(fringe) != 0:
        
        (heuristic, start_city, distance, t_trip, delivery, path) = heapq.heappop(fringe)
        '''
        if start_city == end:
            #return path
            
            d = distance
            t = t_trip
            del_time = delivery
            #path.append((next_city,highway+" for "+str(length)+" miles"))

            return {"total-segments" : len(path), 
            "total-miles" : float(d), 
            "total-hours" : t, 
            "total-delivery-hours" : del_time, 
            "route-taken" : path}
        '''
        #visited_cities.append(start_city)
        #route_taken=[]
        for (next_city,length,speed,highway,t_road,prob) in successors(start_city):
            try:
                lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                h =  get_heuristic(lat1,long1,lat2,long2)
            except:
                h = 0
            
            
            if next_city == end:
                '''
                try:
                    lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                    long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                    h =  get_heuristic(lat1,long1,lat2,long2)
                except:
                    h = 0
                '''
                d = distance + length
                t = t_trip + t_road
                del_time = round(delivery + (t_road + prob*2*(t_trip+t_road)),4)
                path.append((next_city,highway+" for "+str(length)+" miles"))

                return {"total-segments" : len(path), 
                "total-miles" : float(d), 
                "total-hours" : t, 
                "total-delivery-hours" : del_time, 
                "route-taken" : path}
            
            
            if next_city not in visited_cities:
                #print(next_city)
                #segment = len(path)
                #cost_seg = segment + 1
                cost_time = t_trip + h/(road_seg.speed.mean()) #heuristic
                del_time = round(delivery + t_road + prob*2*(t_trip+t_road),4)
                #del_time = round(t_road + prob*2*(t_trip+t_road),4)
                #path.append([next_city,length,speed,highway,t_road,prob])
                movement = (next_city,highway+" for "+str(length)+" miles")
                #path.append([next_city,length,highway,t_road,del_time])
                heapq.heappush(fringe,(cost_time,next_city,distance+length,t_trip+t_road,del_time,path+[movement]))#[(next_city,length,speed,highway,t_road,prob)]))
                visited_cities.append(next_city)
                

In [20]:



################# delivery #################

def route_delivery(start, end, cost):
    fringe = []
    visited_cities = []
    visited_cities.append(start)
    if end in city_gps.city.unique():
        lat2 = city_gps[city_gps.city == end]['lat'].iloc[0]
        long2 = city_gps[city_gps.city == end]['long'].iloc[0]

    heapq.heappush(fringe,(0,start,0,0,0,[])) #no of road seg, curr city, length,time, extra delivery time, path
    while len(fringe) != 0:
        
        (heuristic, start_city, distance, t_trip, delivery, path) = heapq.heappop(fringe)
        '''
        if start_city == end:
            #return path
            
            d = distance
            t = t_trip
            del_time = delivery
            #path.append((next_city,highway+" for "+str(length)+" miles"))

            return {"total-segments" : len(path), 
            "total-miles" : float(d), 
            "total-hours" : t, 
            "total-delivery-hours" : del_time, 
            "route-taken" : path}
        '''
        #visited_cities.append(start_city)
        #route_taken=[]
        for (next_city,length,speed,highway,t_road,prob) in successors(start_city):
            try:
                lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                h =  get_heuristic(lat1,long1,lat2,long2)
            except:
                h = 0
            
            
            if next_city == end:
                '''
                try:
                    lat1 = city_gps[city_gps.city == next_city]['lat'].iloc[0]
                    long1 = city_gps[city_gps.city == next_city]['long'].iloc[0]
                    h =  get_heuristic(lat1,long1,lat2,long2)
                except:
                    h = 0
                '''
                d = distance + length
                t = t_trip + t_road
                del_time = round(delivery + (t_road + prob*2*(t_trip+t_road)),4)
                path.append((next_city,highway+" for "+str(length)+" miles"))

                return {"total-segments" : len(path), 
                "total-miles" : float(d), 
                "total-hours" : t, 
                "total-delivery-hours" : del_time, 
                "route-taken" : path}
            
            
            if next_city not in visited_cities:
                #print(next_city)
                #segment = len(path)
                #cost_seg = segment + 1
                cost_del_time = delivery + prob*(h/(road_seg.speed.mean())) #heuristic
                del_time = round(delivery + t_road + prob*2*(t_trip+t_road),4)
                #del_time = round(t_road + prob*2*(t_trip+t_road),4)
                #path.append([next_city,length,speed,highway,t_road,prob])
                movement = (next_city,highway+" for "+str(length)+" miles")
                #path.append([next_city,length,highway,t_road,del_time])
                heapq.heappush(fringe,(cost_del_time,next_city,distance+length,t_trip+t_road,del_time,path+[movement]))#[(next_city,length,speed,highway,t_road,prob)]))
                visited_cities.append(next_city)
                

In [31]:
7/30 + ((25/52) + np.tanh(25/1000)*2*(25/52 + (7/30+25/52))) + ((19/52) + np.tanh(19/1000)*2*(19/52 + (19/52 + 7/30 + 25/52)))

1.1941168480023647

In [21]:
path=[('Bedford,_Indiana', 'IN_37 for 21 miles'),
  ('Cincinnati,_Indiana', 'IN_45 for 16 miles'),
  ('Columbus,_Indiana', 'IN_46 for 32 miles'),
  ('Martinsville,_Indiana', 'IN_37 for 19 miles')]

In [22]:
for i in [ ('Terre_Haute,_Indiana', 'IN_46 for 35 miles'),
  ('Worthington,_Indiana', 'US_231 for 17 miles')]:
    path+[i]

In [23]:
path

[('Bedford,_Indiana', 'IN_37 for 21 miles'),
 ('Cincinnati,_Indiana', 'IN_45 for 16 miles'),
 ('Columbus,_Indiana', 'IN_46 for 32 miles'),
 ('Martinsville,_Indiana', 'IN_37 for 19 miles')]

In [24]:
p =[]
distance=0
time=0
delivery=0
seg = route('Bloomington,_Indiana', 'Indianapolis,_Indiana','Segments')
for i in seg: #iterating through the popped out goal to get the sum of distance, time and probability of safe path
    distance = distance+i[1]
    time = time + i[3]
    delivery=delivery+ i[4]
    p.append((i[0],str(i[2])+" for "+str(i[1])+" miles"))
    
print(len(seg),distance,time,delivery,p)
#print("total-segments" : len(seg), "total-miles": distance , "total-hours" : time,  "delivery time extra" : delivery ,  "route-taken" :p)

NameError: name 'route' is not defined

In [94]:
2.2151414576207342-1.1364  #-1.07948

1.0787414576207341

In [43]:
road_seg[road_seg.city_2=='Jct_I-465_&_IN_37_S,_Indiana']

,city_1,city_2,length,speed,highway,time,prob_overspeed
7260,"Indianapolis,_Indiana","Jct_I-465_&_IN_37_S,_Indiana",7,30,IN_37,0.23333,0.0


In [37]:
0.23333+ (0.48077+ 2*0.024995*(0.23333+0.48077))+ (0.36538+2*0.018998*(0.23333+0.48077+0.36538))

1.15619378108

In [44]:
(19/52)+2*np.tanh(19/1000)*(19/52) + ((25/52) + np.tanh(25/1000)*2*(25/52 + (19/52))) + 7/30#((19/52) + np.tanh(19/1000)*2*(19/52 + ( 7/30 + 25/52)))

1.1356690047390658

In [39]:
res =route_delivery('Bloomington,_Indiana','Indianapolis,_Indiana','delivery')#Bloomington,_Indiana Indianapolis,_Indiana segments
res

{'total-segments': 3,
 'total-miles': 51.0,
 'total-hours': 1.07948,
 'total-delivery-hours': 1.1357,
 'route-taken': [('Martinsville,_Indiana', 'IN_37 for 19 miles'),
  ('Jct_I-465_&_IN_37_S,_Indiana', 'IN_37 for 25 miles'),
  ('Indianapolis,_Indiana', 'IN_37 for 7 miles')]}

In [26]:
res =route_time('Bloomington,_Indiana', 'Indianapolis,_Indiana', 'time')#Bloomington,_Indiana Indianapolis,_Indiana segments
res

{'total-segments': 3,
 'total-miles': 51.0,
 'total-hours': 1.07948,
 'total-delivery-hours': 1.1357,
 'route-taken': [('Martinsville,_Indiana', 'IN_37 for 19 miles'),
  ('Jct_I-465_&_IN_37_S,_Indiana', 'IN_37 for 25 miles'),
  ('Indianapolis,_Indiana', 'IN_37 for 7 miles')]}

In [27]:
res =route_dist('Bloomington,_Indiana', 'Jct_I-465_&_IN_37_S,_Indiana', 'distance')#Bloomington,_Indiana Indianapolis,_Indiana segments
res

{'total-segments': 2,
 'total-miles': 44.0,
 'total-hours': 0.84615,
 'total-delivery-hours': 0.9024,
 'route-taken': [('Martinsville,_Indiana', 'IN_37 for 19 miles'),
  ('Jct_I-465_&_IN_37_S,_Indiana', 'IN_37 for 25 miles')]}

In [191]:
res =route_seg('Bloomington,_Indiana', 'Jct_I-465_&_IN_37_S,_Indiana', 'segments')#Bloomington,_Indiana Indianapolis,_Indiana segments
res

{'total-segments': 2,
 'total-miles': 44.0,
 'total-hours': 0.84615,
 'total-delivery-hours': 0.9024,
 'route-taken': [('Martinsville,_Indiana', 'IN_37 for 19 miles'),
  ('Jct_I-465_&_IN_37_S,_Indiana', 'IN_37 for 25 miles')]}

In [176]:
p=successors('Martinsville,_Indiana')
p

[['Bloomington,_Indiana', 19, 52, 'IN_37', 0.36538, 0.018997713996764965],
 ['Franklin,_Indiana', 25, 45, 'IN_44', 0.55556, 0.0],
 ['Jct_I-465_&_IN_37_S,_Indiana',
  25,
  52,
  'IN_37',
  0.48077,
  0.02499479296842069],
 ['Jct_IN_67_&_IN_39_S,_Indiana', 3, 30, 'IN_39', 0.1, 0.0]]

In [179]:
city_gps[city_gps['city']=='Indianapolis,_Indiana']

,city,lat,long
2358,"Indianapolis,_Indiana",39.768403,-86.158068


In [ ]:
def calculate_min_heuristic_seg(city_s,city_d):
    cost_segment=1
    visited_node=[]
    fringe_segment = [(0,0,city_s,[])]
    heapq.heapify(fringe_segment)
    lat_d = df_gps.loc[df_gps.city==city_d, 'latitude'].iloc[0] 
    long_d = df_gps.loc[df_gps.city==city_d, 'longitude'].iloc[0]
    lat_s =  df_gps.loc[df_gps.city==city_s, 'latitude'].iloc[0]  
    long_s =  df_gps.loc[df_gps.city==city_s, 'longitude'].iloc[0] 
    while len(fringe_segment) !=0:
        succ = heapq.heappop(fringe_segment) #popping out the path with the least hueristic value
        if succ[2]==city_d: # checking the goal state
            return succ[3]
        visited_node.append(succ[2])  #keeping a track of the visited nodes
        for move in successors(succ[2]):
            try:
                lat_c = df_gps.loc[df_gps.city==move[1], 'latitude'].iloc[0]  
                long_c = df_gps.loc[df_gps.city==move[1], 'longitude'].iloc[0]
                h_s= calculate_heuristic(lat_c,long_c,lat_d,long_d) # heuristic is the haversine distance from current state to goal divided by the average segment length
            except:
                h_s=0  # cathces the case where a certain destination does not have lat and long
            if move[1] not in visited_node:
                cost_segment=len(succ[3])
                # cost + heuristic
                f=cost_segment+h_s/avg_seg
                heapq.heappush(fringe_segment,(f,move[2],move[1],succ[3]+[move]))
                
                
                
                
if cost_fn == "segments":
        seg = calculate_min_heuristic_seg(city_s,city_d)
        for i in seg: #iterating through the popped out goal to get the sum of distance, time and probability of safe path
            distance = distance+i[2]
            time = time + i[5]
            safe=safe+ i[6]
            route.append((i[1],str(i[4])+" for "+str(i[2])+" miles"))
        return{"total-segments" : len(seg), "total-miles": distance , "total-hours" : time,  "total-expected-accidents" : safe ,  "route-taken" : route}

In [ ]:
if (cost_function == "segments"):
		heapq.heappush(heap, (0, start_city, start_city, 0, 0, 0))

		while (len(heap) > 0):
			(segment, start_city, path, distance, time, mpg) = heapq.heappop(heap)
			#print(start_city, "\n")

			for (cities, distance1, time1, mpg1) in successors(start_city):
				if cities == end_city:
					result = (segment+1, cities, path + " " + cities, distance+int(distance1) , round(time+float(time1), 4), round(mpg+mpg1,4))
					return result
				else:
					if cities not in visited_cities:
						heapq.heappush(heap, (segment+1, cities, path + " " + cities, distance+int(distance1) , round(time+float(time1),4), mpg+mpg1))
						visited_cities.append(cities)

In [45]:
road_seg.head()

,city_1,city_2,length,speed,highway,time,prob_overspeed
0,"Y_City,_Arkansas","Acorn,_Arkansas",15,45,US_71/270,0.33333,0.0
1,"Y_City,_Arkansas","Greenwood,_Arkansas",46,45,US_71,1.02222,0.0
2,"Y_City,_Arkansas","Hot_Springs,_Arkansas",70,45,US_270,1.55556,0.0
3,"Abbot_Village,_Maine","Bingham,_Maine",24,45,ME_16,0.53333,0.0
4,"Abbot_Village,_Maine","Guilford,_Maine",4,45,ME_6/15/16,0.08889,0.0


In [56]:
#road_seg['state'] =
x=list(map(lambda x: x.split(',_')[1],road_seg['city_1']))
len(set(x))

74

In [57]:
pwd

'/Users/prataprc94/Desktop/Fall2021 Courses/CSCI-B 551 Elements of AI - 17656/partrao-prroyc-takansal-a1/part2'